# 과제) Cifar10 data를 사용한 모델 성능 향상시키기

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.datasets as datasets
import torchvision.transforms as transforms

import time
import matplotlib.pyplot as plt


Bad key "text.kerning_factor" on line 4 in
C:\Users\82105\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
batch_size = 128
learning_rate = 0.01
epochs=10

<font color='blue'> **데이터 정규화**

In [4]:
standardization = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4913, 0.4821, 0.4465], std = [0.2470, 0.2434, 0.2615])

])

train_dataset = datasets.CIFAR10(root = './data/CIFAR_10', train = True, download = True, transform = standardization)
test_dataset = datasets.CIFAR10(root = './data/CIFAR_10', train = False, download = True, transform = standardization)
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True, drop_last=True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = False, drop_last=True)

0it [00:00, ?it/s]

Extracting ./data/CIFAR_10\cifar-10-python.tar.gz to ./data/CIFAR_10
Files already downloaded and verified


In [5]:
batch_train, batch_labels = next(iter(train_loader))
batch_test, batch_test_labels = next(iter(test_loader))
print(batch_train.shape)
print(batch_labels.shape)

torch.Size([128, 3, 32, 32])
torch.Size([128])


## Build Basic Model

<font color='blue'>**Enhance Model**

    
    1. BatchNormalization : BatchNorm1d  

    2. Activation Function : ReLU      

    3. Initialization : He Initialization  

In [6]:
class cifar10_dnn(nn.Module):
    def __init__(self):
        super(cifar10_dnn, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(3 * 32 * 32, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
        )
        self.layer2 = nn.Sequential(
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
        )
        self.layer4 = nn.Sequential(
            nn.Linear(128, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
        )
        self.layer5 = nn.Sequential(
            nn.Linear(32, 10),
        )
            
    def forward(self, x):
        x = x.view(batch_size, -1)
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        return out

            
model = cifar10_dnn()

def weight_init(m):                      
    if isinstance(m, nn.Linear):                   
        nn.init.kaiming_uniform_(m.weight.data)

model.apply(weight_init)

cifar10_dnn(
  (layer1): Sequential(
    (0): Linear(in_features=3072, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer2): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer3): Sequential(
    (0): Linear(in_features=512, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer4): Sequential(
    (0): Linear(in_features=128, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer5): Sequential(
    (0): Linear(in_features=32, out_features=10, bias=True)
  )
)

<font color='blue'> **모델 테스트**

In [7]:
temp_images = batch_train
test_prediction = model(temp_images)
print(temp_images.shape)
print(test_prediction.shape)

torch.Size([128, 3, 32, 32])
torch.Size([128, 10])


In [8]:
model.to(device)

cifar10_dnn(
  (layer1): Sequential(
    (0): Linear(in_features=3072, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer2): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer3): Sequential(
    (0): Linear(in_features=512, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer4): Sequential(
    (0): Linear(in_features=128, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer5): Sequential(
    (0): Linear(in_features=32, out_features=10, bias=True)
  )
)

In [9]:
optimizer = optim.Adam(model.parameters(), lr = learning_rate)  
loss_func = nn.CrossEntropyLoss()  

In [10]:
loss_arrs = []

def fit(model, train_loader, epochs, optimizer, loss_func):
    for epoch in range(epochs):
        start = time.time()
        avg_loss = 0
        total_batch = len(train_dataset) // batch_size
        
        for num, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = loss_func(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            avg_loss += loss / total_batch
            loss_arrs.append(avg_loss)
        print(f'[Epoch: {epoch+1}/{epochs}]  loss={avg_loss:.4f},  time={time.time()-start:.2f}')
    print('Learning Finished !')

In [11]:
fit(model, train_loader, epochs, optimizer, loss_func)

[Epoch: 1/10]  loss=1.7162,  time=28.82
[Epoch: 2/10]  loss=1.4622,  time=17.86
[Epoch: 3/10]  loss=1.3469,  time=19.82
[Epoch: 4/10]  loss=1.2551,  time=20.23
[Epoch: 5/10]  loss=1.1763,  time=20.56
[Epoch: 6/10]  loss=1.1097,  time=21.23
[Epoch: 7/10]  loss=1.0513,  time=23.00
[Epoch: 8/10]  loss=0.9750,  time=21.04
[Epoch: 9/10]  loss=0.9153,  time=20.42
[Epoch: 10/10]  loss=0.8508,  time=23.28
Learning Finished !


In [12]:
def test(model, test_loader, epochs):
    start = time.time()
    test_acc = 0
    
    for num, (images, labels) in enumerate(test_loader):
        test_images = images.to(device)
        test_labels = labels.to(device)
        
        test_outputs = model(test_images)
        test_batch_acc = ((test_outputs.argmax(dim=1) == test_labels).float().mean()) # acc = 맞춘 개수 / 배치사이즈
        test_acc += test_batch_acc / len(test_loader) # acc / total_Iteration
    print(f'test_acc: {test_acc}, takes {time.time()-start:.2f} secs')

In [13]:
test(model, test_loader, epochs)

test_acc: 0.5522835850715637, takes 3.22 secs


<font color='blue'>**< 5가지 > 기본적으로 가르쳐 준 것들**
    1. activation function -> relu  
    2. weight initialization -> he  
    3. batch normalization -> batchnorm1d  
    4. optimizer -> adam  
    5. dropout -> 정확도가 낮으므로 생략

**< 그 외 추가가능 한 것들 >**
     - data noramlization -> 데이터의 평균, 표준편차 구해서 데이터 자체를 정규화시킴
     - data augmentation -> cnn때 추가, 여기선 생략. 어차피 데이터 조합적인 특징 못 뽑아냄
     - lr_scheduler -> cnn때 추가
